# **Семинар 3 — Создание gRPC-сервиса для ML-модели (protobuf + Python)**

## **Цель занятия**

Освоить базовые навыки проектирования и реализации gRPC-сервиса для ML-модели:  
описать контракт в **Protocol Buffers**, сгенерировать сервер и клиента на Python,  
реализовать эндпоинты `/health` и `/predict`, запустить локальные тесты производительности  
и подготовить структуру проекта к дальнейшей контейнеризации.

---

## **План занятия**

1. Установка инструментов и подготовка окружения  
2. Проектирование контракта API в `.proto`  
3. Генерация Python-кода из protobuf  
4. Реализация сервера gRPC с загрузкой модели  
5. Реализация клиента gRPC и базовых тестов  
6. Валидация входных данных, обработка ошибок, таймауты  
7. Мини-бенчмарк: сравнение с REST (по желанию)  
8. Итог: чек-лист готовности к контейнеризации

---

## **Предварительные требования**

- Python 3.10+  
- Виртуальное окружение (venv или conda)  
- Пакеты: `grpcio`, `grpcio-tools`, `pandas`, `scikit-learn`, `joblib`, `uvloop` (опционально под Linux)  
- Заготовленная модель `model.pkl` (например, LogisticRegression, обученная на Iris или Wine)

---

## **1. Структура проекта (Обновленная)**

Root/

protos/

model.proto

server/

server.py

inference.py

validation.py

client/

client.py

models/

model.pkl

requirements.txt

Makefile

README.md

In [1]:
# Установка необходимых библиотек
!pip install grpcio==1.66.1 grpcio-tools==1.66.1 pandas scikit-learn joblib uvloop -q


## **2. Создание структуры проекта**

Создадим базовую структуру папок для gRPC-сервиса в текущей директории.



In [2]:
import os

# Обновленные пути (без префикса ml_grpc_service)
folders = [
    "protos",
    "server",
    "client",
    "models"
]
for f in folders:
    os.makedirs(f, exist_ok=True)

print("✅ Структура каталогов создана.")
!tree .


✅ Структура каталогов создана.
zsh:1: command not found: tree


## **3. Проектирование контракта API — `model.proto`**

Файл `.proto` описывает, какие методы и типы данных доступны клиенту и серверу.  
Ниже — минимальный, но расширяемый контракт с двумя RPC-методами: **Health** и **Predict**.  
Добавлено поле `model_version` и возможность передавать произвольные числовые признаки в виде повторяющегося поля.


In [3]:
proto_code = """
syntax = "proto3";
package mlservice.v1;

service PredictionService {
  rpc Health(HealthRequest) returns (HealthResponse);
  rpc Predict(PredictRequest) returns (PredictResponse);
}

message HealthRequest {}

message HealthResponse {
  string status = 1;         // "ok"
  string model_version = 2;  // e.g. "v1.0.3"
}

message Feature {
  string name = 1;
  double value = 2;
}

message PredictRequest {
  repeated Feature features = 1;  // [{name:"sepal_length", value:5.1}, ...]
}

message PredictResponse {
  string prediction = 1;          // "setosa"
  double confidence = 2;          // 0.93 (опционально)
  string model_version = 3;
}
"""

with open("protos/model.proto", "w") as f:
    f.write(proto_code.strip())

print("✅ Файл model.proto создан:")
!cat protos/model.proto


✅ Файл model.proto создан:
syntax = "proto3";
package mlservice.v1;

service PredictionService {
  rpc Health(HealthRequest) returns (HealthResponse);
  rpc Predict(PredictRequest) returns (PredictResponse);
}

message HealthRequest {}

message HealthResponse {
  string status = 1;         // "ok"
  string model_version = 2;  // e.g. "v1.0.3"
}

message Feature {
  string name = 1;
  double value = 2;
}

message PredictRequest {
  repeated Feature features = 1;  // [{name:"sepal_length", value:5.1}, ...]
}

message PredictResponse {
  string prediction = 1;          // "setosa"
  double confidence = 2;          // 0.93 (опционально)
  string model_version = 3;
}

## **4. Генерация Python-кода из protobuf**

Теперь скомпилируем `.proto` в Python-код.  
Команда `protoc` из пакета `grpcio-tools` создаёт два файла:
- `model_pb2.py` — описания сообщений;
- `model_pb2_grpc.py` — интерфейс сервиса.

Мы генерируем файлы прямо в корень проекта (root), чтобы упростить импорты.


In [4]:
# Генерация файлов в текущую директорию (.)
!python -m grpc_tools.protoc -I. \
  --python_out=. \
  --grpc_python_out=. \
  protos/model.proto

print("✅ Код сгенерирован:")
!ls | grep model


✅ Код сгенерирован:
models


## **5. Логика инференса и валидации данных**

### **5.1 inference.py**

Файл `server/inference.py` отвечает за загрузку модели и выполнение предсказаний.  
Для примера используется модель `model.pkl`, обученная заранее (например, LogisticRegression на Iris).  
Метод `predict()` возвращает предсказанный класс и confidence (если доступен `predict_proba`).


In [5]:
inference_code = """
import joblib
import pandas as pd
from pathlib import Path

class ModelRunner:
    def __init__(self, model_path: str, version: str = "v1.0.0"):
        self.model = joblib.load(model_path)
        self.version = version

    def predict(self, features: dict[str, float]) -> tuple[str, float]:
        df = pd.DataFrame([features])
        y = self.model.predict(df)[0]
        try:
            proba = float(max(self.model.predict_proba(df)[0]))
        except Exception:
            proba = 1.0
        return str(y), proba
"""

with open("server/inference.py", "w") as f:
    f.write(inference_code.strip())

print("✅ inference.py создан.")
!head -n 20 server/inference.py


✅ inference.py создан.
import joblib
import pandas as pd
from pathlib import Path

class ModelRunner:
    def __init__(self, model_path: str, version: str = "v1.0.0"):
        self.model = joblib.load(model_path)
        self.version = version

    def predict(self, features: dict[str, float]) -> tuple[str, float]:
        df = pd.DataFrame([features])
        y = self.model.predict(df)[0]
        try:
            proba = float(max(self.model.predict_proba(df)[0]))
        except Exception:
            proba = 1.0
        return str(y), proba

### **5.2 validation.py**

Файл `server/validation.py` отвечает за проверку входных данных:
- отсутствие дубликатов признаков,
- отсутствие пустых имён,
- наличие хотя бы одного признака.

При ошибке выбрасывается исключение `ValidationError`.


In [6]:
validation_code = """
from typing import Iterable
import model_pb2  # Import from root

class ValidationError(Exception):
    pass

def features_to_dict(features: Iterable[model_pb2.Feature]) -> dict[str, float]:
    data = {}
    for f in features:
        if f.name in data:
            raise ValidationError(f"Duplicate feature: {f.name}")
        if not f.name:
            raise ValidationError("Empty feature name")
        data[f.name] = float(f.value)
    if not data:
        raise ValidationError("No features provided")
    return data
"""

with open("server/validation.py", "w") as f:
    f.write(validation_code.strip())

print("✅ validation.py создан.")
!head -n 20 server/validation.py


✅ validation.py создан.
from typing import Iterable
import model_pb2  # Import from root

class ValidationError(Exception):
    pass

def features_to_dict(features: Iterable[model_pb2.Feature]) -> dict[str, float]:
    data = {}
    for f in features:
        if f.name in data:
            raise ValidationError(f"Duplicate feature: {f.name}")
        if not f.name:
            raise ValidationError("Empty feature name")
        data[f.name] = float(f.value)
    if not data:
        raise ValidationError("No features provided")
    return data

## **6. Реализация сервера gRPC**

Сервер принимает запросы `/Health` и `/Predict`, выполняет валидацию входных данных,  
делает предсказание через `ModelRunner` и возвращает результат клиенту.  
Ошибки передаются через `context.set_code` и `context.set_details`.


In [7]:
server_code = """
import grpc, os
from concurrent import futures
import model_pb2, model_pb2_grpc  # Imports from root
from server.inference import ModelRunner
from server.validation import features_to_dict, ValidationError

MODEL_PATH = os.getenv("MODEL_PATH", "models/model.pkl")
MODEL_VERSION = os.getenv("MODEL_VERSION", "v1.0.0")
MAX_WORKERS = int(os.getenv("MAX_WORKERS", "4"))
PORT = int(os.getenv("PORT", "50051"))

class PredictionService(model_pb2_grpc.PredictionServiceServicer):
    def __init__(self):
        self.runner = ModelRunner(MODEL_PATH, version=MODEL_VERSION)

    def Health(self, request, context):
        return model_pb2.HealthResponse(status="ok", model_version=self.runner.version)

    def Predict(self, request, context):
        try:
            feats = features_to_dict(request.features)
            pred, conf = self.runner.predict(feats)
            return model_pb2.PredictResponse(
                prediction=pred, confidence=conf, model_version=self.runner.version
            )
        except ValidationError as ve:
            context.set_code(grpc.StatusCode.INVALID_ARGUMENT)
            context.set_details(str(ve))
            return model_pb2.PredictResponse()
        except Exception as e:
            context.set_code(grpc.StatusCode.INTERNAL)
            context.set_details(f"internal error: {e}")
            return model_pb2.PredictResponse()

def serve():
    options = [
        ("grpc.max_send_message_length", 50 * 1024 * 1024),
        ("grpc.max_receive_message_length", 50 * 1024 * 1024),
    ]
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=MAX_WORKERS), options=options)
    model_pb2_grpc.add_PredictionServiceServicer_to_server(PredictionService(), server)
    server.add_insecure_port(f"[::]:{PORT}")
    server.start()
    print(f"gRPC server started on :{PORT}, model={MODEL_PATH}, version={MODEL_VERSION}")
    server.wait_for_termination()

if __name__ == "__main__":
    try:
        import uvloop
        uvloop.install()
    except Exception:
        pass
    serve()
"""

with open("server/server.py", "w") as f:
    f.write(server_code.strip())

print("✅ server.py создан.")
!head -n 25 server/server.py


✅ server.py создан.
import grpc, os
from concurrent import futures
import model_pb2, model_pb2_grpc  # Imports from root
from server.inference import ModelRunner
from server.validation import features_to_dict, ValidationError

MODEL_PATH = os.getenv("MODEL_PATH", "models/model.pkl")
MODEL_VERSION = os.getenv("MODEL_VERSION", "v1.0.0")
MAX_WORKERS = int(os.getenv("MAX_WORKERS", "4"))
PORT = int(os.getenv("PORT", "50051"))

class PredictionService(model_pb2_grpc.PredictionServiceServicer):
    def __init__(self):
        self.runner = ModelRunner(MODEL_PATH, version=MODEL_VERSION)

    def Health(self, request, context):
        return model_pb2.HealthResponse(status="ok", model_version=self.runner.version)

    def Predict(self, request, context):
        try:
            feats = features_to_dict(request.features)
            pred, conf = self.runner.predict(feats)
            return model_pb2.PredictResponse(
                prediction=pred, confidence=conf, model_version=self.runner.v

## **7. Клиент gRPC и ручные прогоны**

Реализуем клиента, который умеет:
- создавать `stub` для подключения к серверу;
- вызывать `Health` с таймаутом;
- вызывать `Predict` с набором признаков Iris.


In [8]:
client_code = """
import grpc
import model_pb2, model_pb2_grpc  # Imports from root

def make_stub(addr: str = "localhost:50051"):
    channel = grpc.insecure_channel(addr)
    return model_pb2_grpc.PredictionServiceStub(channel)

def health(stub):
    res = stub.Health(model_pb2.HealthRequest(), timeout=2.0)
    print("Health:", res.status, "version:", res.model_version)

def predict(stub):
    req = model_pb2.PredictRequest(features=[
        model_pb2.Feature(name="sepal_length", value=5.1),
        model_pb2.Feature(name="sepal_width", value=3.5),
        model_pb2.Feature(name="petal_length", value=1.4),
        model_pb2.Feature(name="petal_width", value=0.2),
    ])
    res = stub.Predict(req, timeout=3.0)
    print("Prediction:", res.prediction, "confidence:", round(res.confidence, 4), "version:", res.model_version)

if __name__ == "__main__":
    stub = make_stub()
    health(stub)
    predict(stub)
"""
with open("client/client.py", "w") as f:
    f.write(client_code.strip())

print("✅ client.py создан.")
!sed -n '1,120p' client/client.py


✅ client.py создан.
import grpc
import model_pb2, model_pb2_grpc  # Imports from root

def make_stub(addr: str = "localhost:50051"):
    channel = grpc.insecure_channel(addr)
    return model_pb2_grpc.PredictionServiceStub(channel)

def health(stub):
    res = stub.Health(model_pb2.HealthRequest(), timeout=2.0)
    print("Health:", res.status, "version:", res.model_version)

def predict(stub):
    req = model_pb2.PredictRequest(features=[
        model_pb2.Feature(name="sepal_length", value=5.1),
        model_pb2.Feature(name="sepal_width", value=3.5),
        model_pb2.Feature(name="petal_length", value=1.4),
        model_pb2.Feature(name="petal_width", value=0.2),
    ])
    res = stub.Predict(req, timeout=3.0)
    print("Prediction:", res.prediction, "confidence:", round(res.confidence, 4), "version:", res.model_version)

if __name__ == "__main__":
    stub = make_stub()
    health(stub)
    predict(stub)

## **8. Подготовка модели `model.pkl`**

Для демонстрации обучим простую `LogisticRegression` на Iris и сохраним в `models/model.pkl`.


In [9]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
import pandas as pd, joblib, os

# Load Iris
iris = datasets.load_iris(as_frame=True)
df = iris.frame

# Rename columns to match what client sends
X = df[iris.feature_names].rename(columns={
    'sepal length (cm)': 'sepal_length',
    'sepal width (cm)': 'sepal_width', 
    'petal length (cm)': 'petal_length',
    'petal width (cm)': 'petal_width'
})
y = df['target']

# Train model
model = LogisticRegression(max_iter=500)
model.fit(X, y)

# Save
os.makedirs("ml_grpc_service/models", exist_ok=True)
joblib.dump(model, "ml_grpc_service/models/model.pkl")
print("✅ Model saved with correct feature names")

✅ Model saved with correct feature names


## **9. Запуск сервера gRPC**

Поднимем сервер в фоне и посмотрим его лог.  
По умолчанию: `PORT=50051`, `MODEL_PATH=models/model.pkl`.


In [10]:
import os, subprocess, time, sys, signal

env = os.environ.copy()
env["PYTHONPATH"] = os.getcwd()  # чтобы пакеты model_pb2 корректно импортировались
# при желании можно эмулировать задержку предсказания:
# env["SLEEP_MS"] = "0"

server_proc = subprocess.Popen(
    [sys.executable, "-m", "server.server"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

time.sleep(1.2)  # дать серверу стартовать

# показать первые строки лога
for _ in range(4):
    line = server_proc.stdout.readline().strip()
    if line:
        print(line)


Traceback (most recent call last):
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/Users/katrindar/MIPT/Masters/ml_grpc_service/server/server.py", line 3, in <module>


## **10. Проверка Health и Predict**

Вызовем методы через клиента: `/Health` и `/Predict`.


In [14]:
import runpy, os, sys

# Запуск клиента как модуля
os.environ["PYTHONPATH"] = os.getcwd()
runpy.run_path("client/client.py")


{'__name__': '<run_path>',
 '__doc__': None,
 '__package__': '',
 '__loader__': None,
 '__spec__': None,
 '__file__': 'client/client.py',
 '__cached__': None,
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import__(name, globals=None, locals=None, fromlist=(), level=0)>,
  'abs': <function

## **11. Негативные сценарии: валидация и таймауты**

1) Попробуем отправить **дубликат признака** — сервер должен вернуть `INVALID_ARGUMENT`.  
2) Эмулируем **таймаут** клиента: включим задержку на сервере через переменную окружения `SLEEP_MS`.


In [22]:
# 11.1 Дубликат признака -> INVALID_ARGUMENT
import grpc
from protos import model_pb2, model_pb2_grpc

stub = model_pb2_grpc.PredictionServiceStub(grpc.insecure_channel("localhost:50051"))

bad_req = model_pb2.PredictRequest(features=[
    model_pb2.Feature(name="sepal_length", value=5.1),
    model_pb2.Feature(name="sepal_length", value=5.2),  # дубликат
])
try:
    _ = stub.Predict(bad_req, timeout=3.0)
except grpc.RpcError as e:
    print("Duplicate feature -> status:", e.code().name, "| details:", e.details())

# 11.2 Таймаут клиента: перезапустим сервер с задержкой
import os, subprocess, sys, time, signal

# остановим текущий сервер
server_proc.send_signal(signal.SIGINT)
server_proc.wait(timeout=3)

env = os.environ.copy()
env["PYTHONPATH"] = os.getcwd()
env["SLEEP_MS"] = "500"  # 0.5 секунды задержка на предсказание

server_proc = subprocess.Popen(
    [sys.executable, "-m", "server.server"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)
time.sleep(1.2)

# попробуем вызвать с слишком маленьким timeout
try:
    _ = stub.Predict(
        model_pb2.PredictRequest(features=[
            model_pb2.Feature(name="sepal_length", value=5.1),
            model_pb2.Feature(name="sepal_width", value=3.5),
            model_pb2.Feature(name="petal_length", value=1.4),
            model_pb2.Feature(name="petal_width", value=0.2),
        ]),
        timeout=0.2,  # 200 мс < задержки 500 мс
    )
except grpc.RpcError as e:
    print("Timeout test -> status:", e.code().name, "| details:", e.details())


Duplicate feature -> status: INVALID_ARGUMENT | details: Duplicate feature: sepal_length
Timeout test -> status: UNAVAILABLE | details: Socket closed


## **12. Мини-бенчмарк: 100 последовательных предсказаний**

Сделаем простой прогон 100 RPC-вызовов `Predict` и измерим суммарное время.  
(Для корректных результатов установите `SLEEP_MS=0` или не задавайте переменную.)


In [23]:
# перезапустим сервер без задержки
import signal, time, os, sys, subprocess

server_proc.send_signal(signal.SIGINT)
server_proc.wait(timeout=3)

env = os.environ.copy()
env["PYTHONPATH"] = os.getcwd()
env.pop("SLEEP_MS", None)

server_proc = subprocess.Popen(
    [sys.executable, "-m", "server.server"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)
time.sleep(1.2)

# бенчмарк
import grpc, time
from protos import model_pb2, model_pb2_grpc

stub = model_pb2_grpc.PredictionServiceStub(grpc.insecure_channel("localhost:50051"))

req = model_pb2.PredictRequest(features=[
    model_pb2.Feature(name="sepal_length", value=5.1),
    model_pb2.Feature(name="sepal_width", value=3.5),
    model_pb2.Feature(name="petal_length", value=1.4),
    model_pb2.Feature(name="petal_width", value=0.2),
])

t0 = time.time()
N = 100
for _ in range(N):
    _ = stub.Predict(req, timeout=2.0)
t1 = time.time()

print(f"{N} Predict RPCs in {t1 - t0:.3f} s, avg {(t1 - t0)/N*1000:.2f} ms/call")


100 Predict RPCs in 0.080 s, avg 0.80 ms/call


## **13. Обновление сервера для опциональной задержки (для тестов таймаута)**

Если вы хотите оставить поддержку переменной `SLEEP_MS`, обновите `server.py`,  
добавив внутри `Predict` небольшой `sleep`. Это удобно для учебных тестов таймаутов.


In [24]:
# Патчим server.py: добавим поддержку задержки через env SLEEP_MS
from pathlib import Path

p = Path("server/server.py")
code = p.read_text()

if "SLEEP_MS" not in code:
    code = code.replace(
        "def Predict(self, request, context):",
        "def Predict(self, request, context):\n        import os, time\n        sleep_ms = int(os.getenv('SLEEP_MS', '0'))\n        if sleep_ms > 0:\n            time.sleep(sleep_ms / 1000.0)"
    )
    p.write_text(code)

print("✅ server.py обновлён (SLEEP_MS поддерживается).")
!sed -n '1,140p' server/server.py


✅ server.py обновлён (SLEEP_MS поддерживается).
import grpc, os
from concurrent import futures
from protos import model_pb2, model_pb2_grpc  # Imports from root
from server.inference import ModelRunner
from server.validation import features_to_dict, ValidationError

MODEL_PATH = os.getenv("MODEL_PATH", "models/model.pkl")
MODEL_VERSION = os.getenv("MODEL_VERSION", "v1.0.0")
MAX_WORKERS = int(os.getenv("MAX_WORKERS", "4"))
PORT = int(os.getenv("PORT", "50051"))

class PredictionService(model_pb2_grpc.PredictionServiceServicer):
    def __init__(self):
        self.runner = ModelRunner(MODEL_PATH, version=MODEL_VERSION)

    def Health(self, request, context):
        return model_pb2.HealthResponse(status="ok", model_version=self.runner.version)

    def Predict(self, request, context):
        import os, time
        sleep_ms = int(os.getenv('SLEEP_MS', '0'))
        if sleep_ms > 0:
            time.sleep(sleep_ms / 1000.0)
        try:
            feats = features_to_dict(request.fea

## **14. Типовые ошибки и отладка**

- **Импорты `model_pb2` не находятся.** Проверьте, куда `protoc` вывел файлы и корректно ли выставлен `PYTHONPATH`.  
- **Сервер не видит модель.** Убедитесь, что `MODEL_PATH` указывает на реальный `model.pkl` и текущая рабочая директория верна.  
- **Клиент «висит».** Всегда задавайте `timeout` в RPC-вызовах.  
- **Изменили `.proto`, а код не обновили.** Регенерируйте Python-модули `model_pb2*.py` на сервере и клиенте.


## **15. Чек-лист готовности к контейнеризации**

- Контракт в `.proto` описан и хранится в VCS.  
- Python-код сгенерирован автоматикой (`grpcio-tools`) и не редактируется вручную.  
- Сервер поднимается, `/Health` возвращает `ok` и `model_version`.  
- `/Predict` работает, ошибки валидируются (дубликаты, пустые признаки).  
- Клиент использует таймауты и реализует сценарии «счастливый путь» и негативные кейсы.  
- Проект структурирован; зависимости закреплены в `requirements.txt`.  
- Параметры (`PORT`, `MODEL_PATH`, `MODEL_VERSION`, опционально `SLEEP_MS`) вынесены в переменные окружения.  
- Готово к контейнеризации (Dockerfile + `ENTRYPOINT` → следующий семинар).